<div align='center' style='font-size:300%'>Validating Computational Models with</div>
<img src="https://raw.githubusercontent.com/scidash/assets/master/logos/SciUnit/sci-unit-wide.png" width="25%">
<hr>
<div align='center' style='font-size:200%; line-height:30px;'>
Richard C. (Rick) Gerkin, PhD
</div>    
<div align='center' style='font-size:150%; line-height:30px;'>Associate Research Professor, School of Life Sciences<br>
Co-Director, <a href="http://iconlab.asu.edu">Laboratory for Informatics and Computation in Open Neuroscience</a><br>
Arizona State University, Tempe, AZ USA
</div>
<hr>

# SciUnit is a framework for validating scientific models by creating experimental-data-driven unit tests.  

# Chapter 1. What is SciUnit?
Everyone hopes that their model has some correspondence with reality.  Usually, checking whether this is true is done informally.
### SciUnit makes this formal and transparent.  

<hr>
<div align='center' style='font-size:300%; line-height:30px;'>
Links:
</div>
<table align='center' style='font-size:150%; line-height:30px;'>
<tr>
    <td width='25%'><img src="https://github.com/scidash/assets/blob/master/logos/SciUnit/sci-unit-wide.png?raw=true"  href="http://sciunit.scidash.org" width="50%"></td>
    <td width='25%'><img src="https://github.com/scidash/assets/blob/master/logos/neuronunit-logo.png?raw=trueg" href="http://neuronunit.scidash.org" width="50%"></td>
</tr>
<tr>
    <td width='25%'><img src="https://github.com/scidash/assets/blob/master/logos/scidash_logo.png?raw=true"  href="http://dash.scidash.org" width="50%"></td>
    <td width='25%'><img src="http://science-marketplace.org/oc-content/uploads/6/599.png" width="35%"></td>
</tr>
</table>

<hr>
<div align='center' style='font-size:200%; line-height:30px;'>
Funded by:
</div>
<table align='center' style='font-size:150%; line-height:30px;'>
<tr>
    <td width='25%'>R01DC018455 (NIDCD)</td>
    <td width='25%'><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/US-NIH-NIDCD-Logo.svg/1920px-US-NIH-NIDCD-Logo.svg.png" width="50%"></td>
    <td width='25%'>R01MH106674 (NIMH)</td>
    <td width='25%'><img src="https://upload.wikimedia.org/wikipedia/commons/a/a0/NIH-NIMH-logo-new.png" width="100%"></td>
</tr>
<tr>
    <td>R01EB021711 (NIBIB)</td>
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/1/15/NIH_NIBIB_Vertical_Logo_2Color.jpg" width="50%"></td>
    <td>Human Brain Project</td>
    <td><img src="https://pbs.twimg.com/profile_images/660035391442042880/v7RkSosC_400x400.png" width="50%"></td>
</tr>
</table>